## Fandom Radius National Graph

In [140]:
import pandas as pd
import numpy as np
import plotly.express as px


In [141]:
# Load all team radii
radii_df = pd.read_csv("/Users/elisabethkollrack/Thesis/EK-Thesis/Fandom Radii/fandom_radius_all_teams.csv")

In [142]:
# Geodesic circle function
def geodesic_circle(lat, lon, radius_km, n_points=360):
    earth_radius_km = 6371.0
    angles = np.linspace(0, 2*np.pi, n_points)

    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)

    circle_lats = np.arcsin(
        np.sin(lat_rad) * np.cos(radius_km / earth_radius_km) +
        np.cos(lat_rad) * np.sin(radius_km / earth_radius_km) * np.cos(angles)
    )

    circle_lons = lon_rad + np.arctan2(
        np.sin(angles) * np.sin(radius_km / earth_radius_km) * np.cos(lat_rad),
        np.cos(radius_km / earth_radius_km) - np.sin(lat_rad) * np.sin(circle_lats)
    )

    return np.degrees(circle_lats), np.degrees(circle_lons)

In [143]:
overall_df = radii_df[radii_df["season"] == "overall"].copy()

fig = px.scatter_geo(
    overall_df,
    lat="lat",
    lon="lon",
    hover_name="city",
    scope="usa",
    title="NFL Fandom Radii Across the United States (2011–2014 Overall)"
)

for _, row in overall_df.iterrows():
    lats, lons = geodesic_circle(row["lat"], row["lon"], row["radius_km"])

    trace = px.line_geo(lat=lats, lon=lons).data[0]

    trace.update(
        hovertemplate=(
            f"<b>{row['team_name']}</b><br>"
            f"Fandom radius ≈ {int(row['radius_km'])} km"
            "<extra></extra>"
        ),
        name=row["team_name"],
        showlegend=False,
        opacity=0.4
    )

    fig.add_trace(trace)

    fig.update_geos(
    scope="usa",
    showland=True,
    landcolor="lightgray",
    showocean=True,
    oceancolor="azure",
    showlakes=True,
    lakecolor="azure"
)

fig.update_layout(title_x=0.5)
fig.show()



In [144]:
fig_hist = px.bar(
    overall_df.sort_values("radius_km", ascending=True),
    x="radius_km",
    y="team_name",
    orientation="h",
    title="NFL Fandom Radii by Team (2011–2014)",
    labels={"R_km": "Fandom Radius (km)", "team": "NFL Team"}
)

fig_hist.update_layout(
    title_x=0.5,
    margin=dict(l=220, t=80),
    font=dict(size=10)
)

fig_hist.update_yaxes(
    tickfont=dict(size=9),
    type="category",
    tickmode="linear"
)

fig_hist.show()


In [145]:
overall_df = radii_df[radii_df["season"] == "2011"].copy()

fig = px.scatter_geo(
    overall_df,
    lat="lat",
    lon="lon",
    hover_name="city",
    scope="usa",
    title="NFL Fandom Radii Across the United States (2011)"
)

for _, row in overall_df.iterrows():
    lats, lons = geodesic_circle(row["lat"], row["lon"], row["radius_km"])

    trace = px.line_geo(lat=lats, lon=lons).data[0]

    trace.update(
        hovertemplate=(
            f"<b>{row['team_name']}</b><br>"
            f"Fandom radius ≈ {int(row['radius_km'])} km"
            "<extra></extra>"
        ),
        name=row["team_name"],
        showlegend=False,
        opacity=0.4
    )

    fig.add_trace(trace)

    fig.update_geos(
    scope="usa",
    showland=True,
    landcolor="lightgray",
    showocean=True,
    oceancolor="azure",
    showlakes=True,
    lakecolor="azure"
)

fig.update_layout(title_x=0.5)
fig.show()



In [146]:
# data frame for all teams in 2011-2014
all_teams_df = radii_df[radii_df["season"].isin(["2011", "2012", "2013", "2014"])].copy()
pivot_df = all_teams_df.pivot(index="team_name", columns="season", values="radius_km").reset_index()
pivot_df.columns.name = None
pivot_df

,team_name,2011,2012,2013,2014
0,Arizona Cardinals,410.006940,408.559240,185.158045,419.022225
1,Atlanta Falcons,189.429786,210.125861,162.424329,231.082727
2,Baltimore Ravens,86.843071,129.066077,186.257740,144.625263
3,Buffalo Bills,283.760203,270.041225,285.169895,252.915557
4,Carolina Panthers,461.625087,299.643127,284.090190,215.501178
5,Chicago Bears,374.081156,196.142166,264.865947,174.110660
6,Cincinnati Bengals,142.779032,360.958569,298.573767,345.506262
7,Cleveland Browns,474.154207,179.802759,164.626664,161.779174
8,Dallas Cowboys,306.196933,99.367923,113.979803,358.324535
9,Denver Broncos,533.350917,533.330339,573.918955,750.152074


In [147]:
# Melt pivot_df to long format
long_df = pivot_df.melt(id_vars="team_name", var_name="Season", value_name="Radius_km")

# Plot with Plotly Express
fig_changes = px.line(
    long_df,
    x="Season",
    y="Radius_km",
    color="team_name",
    markers=True,  # adds points at each season
    title="NFL Fandom Radii Changes by Team (2011–2014)",
    labels={"Radius_km": "Fandom Radius (km)", "team_name": "NFL Team"}
)

fig_changes.update_layout(title_x=0.5)
fig_changes.show()


In [148]:
# save as interactive plot
fig_changes.write_html("nfl_fandom_radius.html")


In [149]:
# import win percentages
win_pct_df = pd.read_csv("/Users/elisabethkollrack/Thesis/EK-Thesis/team_season_win_pct.csv")

# print ARI win percentages
ari_win_pct = win_pct_df[win_pct_df["team"] == "ATL"]
print(ari_win_pct)

     season team  games  wins   win_pct
1      2010  ATL     17    13  0.764706
33     2011  ATL     17    10  0.588235
65     2012  ATL     18    14  0.777778
97     2013  ATL     16     4  0.250000
129    2014  ATL     16     6  0.375000


In [150]:
radii_df

,team_name,city,season,lat,lon,radius_km,n_tweets,n_metros
0,Arizona Cardinals,Phoenix,2011,33.4483,-112.0725,410.006940,369,92
1,Arizona Cardinals,Phoenix,2012,33.4483,-112.0725,408.559240,463,116
2,Arizona Cardinals,Phoenix,2013,33.4483,-112.0725,185.158045,732,149
3,Arizona Cardinals,Phoenix,2014,33.4483,-112.0725,419.022225,609,121
4,Arizona Cardinals,Phoenix,overall,33.4483,-112.0725,411.502044,2173,233
...,...,...,...,...,...,...,...,...
145,Washington Redskins,"Washington, D.C.",2011,38.9073,-77.0369,194.129912,381,77
146,Washington Redskins,"Washington, D.C.",2012,38.9073,-77.0369,109.997030,1480,175
147,Washington Redskins,"Washington, D.C.",2013,38.9073,-77.0369,221.667657,1249,195
148,Washington Redskins,"Washington, D.C.",2014,38.9073,-77.0369,122.462912,1540,184


In [151]:
# --- Clean season column: keep only numeric seasons ---
radii_df["season"] = pd.to_numeric(radii_df["season"], errors="coerce")
win_pct_df["season"] = pd.to_numeric(win_pct_df["season"], errors="coerce")

# drop rows like "overall"
radii_df = radii_df.dropna(subset=["season"])
win_pct_df = win_pct_df.dropna(subset=["season"])

radii_df["season"] = radii_df["season"].astype(int)
win_pct_df["season"] = win_pct_df["season"].astype(int)

# --- Map team abbreviations to full team names ---
team_map = {
    "ARI": "Arizona Cardinals", "ATL": "Atlanta Falcons", "BAL": "Baltimore Ravens",
    "BUF": "Buffalo Bills", "CAR": "Carolina Panthers", "CHI": "Chicago Bears",
    "CIN": "Cincinnati Bengals", "CLE": "Cleveland Browns", "DAL": "Dallas Cowboys",
    "DEN": "Denver Broncos", "DET": "Detroit Lions", "GB": "Green Bay Packers",
    "HOU": "Houston Texans", "IND": "Indianapolis Colts", "JAX": "Jacksonville Jaguars",
    "KC": "Kansas City Chiefs", "LV": "Las Vegas Raiders", "OAK": "Oakland Raiders",
    "MIA": "Miami Dolphins", "MIN": "Minnesota Vikings", "NE": "New England Patriots",
    "NO": "New Orleans Saints", "NYG": "New York Giants", "NYJ": "New York Jets",
    "PHI": "Philadelphia Eagles", "PIT": "Pittsburgh Steelers", "SEA": "Seattle Seahawks",
    "SF": "San Francisco 49ers", "TB": "Tampa Bay Buccaneers", "TEN": "Tennessee Titans",
    "WAS": "Washington Redskins"
}

win_pct_df["team_name"] = win_pct_df["team"].map(team_map)

# --- Filter to 2011–2014 ---
radii_sub = radii_df[radii_df["season"].isin([2011, 2012, 2013, 2014])]
win_pct_sub = win_pct_df[win_pct_df["season"].isin([2011, 2012, 2013, 2014])]

# --- Merge ---
merged_df = pd.merge(
    radii_sub,
    win_pct_sub,
    on=["team_name", "season"],
    how="inner"
)

# --- Correlation ---
correlation = merged_df["radius_km"].corr(merged_df["win_pct"])
print(f"Correlation between fandom radius and win percentage: {correlation:.4f}")

# --- Scatter plot ---
fig_corr = px.scatter(
    merged_df,
    x="radius_km",
    y="win_pct",
    hover_name="team_name",
    hover_data={"season": True, "win_pct": ":.3f", "radius_km": ":.1f"},
    trendline="ols",
    title="NFL Fandom Radius vs Win Percentage (2011–2014)",
    labels={"win_pct": "Win Percentage", "radius_km": "Fandom Radius (km)"}
)


fig_corr.update_layout(title_x=0.5)
fig_corr.show()


Correlation between fandom radius and win percentage: 0.1686


/var/folders/cg/kg987wfn3_977vnx3pkwqg0m0000gn/T/ipykernel_34125/267684539.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [152]:
# save as interactive plot
fig_corr.write_html("nfl_fandom_radius_vs_win_percentage.html")


In [159]:
win_pct_lag = win_pct_df.copy()
win_pct_lag["season"] = win_pct_lag["season"] + 1
win_pct_lag = win_pct_lag.rename(columns={"win_pct": "win_pct_prev"})

lag_merged_df = pd.merge(
    radii_df,
    win_pct_lag,
    on=["team_name", "season"],
    how="inner"
)

lag_merged_df = lag_merged_df[lag_merged_df["season"].isin([2011, 2012, 2013, 2014])]

fig_lag = px.scatter(
    lag_merged_df,
    y="win_pct_prev",
    x="radius_km",
    hover_name="team_name",
    hover_data={"season": True, "win_pct_prev": ":.3f", "radius_km": ":.1f"},
    trendline="ols",
    title="Previous Season Win % vs Current Season Fandom Radius (2011–2014)",
    labels={
        "win_pct_prev": "Previous Season Win Percentage",
        "radius_km": "Current Season Fandom Radius (km)"
    }
)

fig_lag.update_layout(title_x=0.5)
fig_lag.show()

lag_corr = lag_merged_df["radius_km"].corr(lag_merged_df["win_pct_prev"])
print(f"Lagged correlation (prev win % vs current radius): {lag_corr:.4f}")



Lagged correlation (prev win % vs current radius): 0.1062


In [160]:
fig_lag.write_html("nfl_fandom_radius_vs_prev_win_percentage.html")